# Task 1: Simple Chat Conversation Loop

This task builds a simple chat conversation loop that allows the chatbot to interact with the user continuously. We extend the basic conversation loop to build a simple working chat application using the OpenAI model and a message list.

In [ ]:
# Import modules
# ==============

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from dotenv import load_dotenv
import os

# Initialize variables
# ===================

# Initialize the ChatOpenAI model with appropriate parameters
load_dotenv()

llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

# Initialize the message list with a system message
messages = [
    SystemMessage(
        content="""You are an expert in handling weather data.
        You should help the user with their weather-related queries
        Use the functions defined as and when needed.
        The name of this app is 'SimpleChatBot'.""")
]

print("🤖 SimpleChatBot initialized!")
print("Type 'exit' to end the conversation.")
print("-" * 50)

# Conversation Loop
# ================

while True:
    # Get user input
    user_input = input("\nYou: ")
    
    # Check if user wants to exit
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("\n👋 Goodbye! Thanks for chatting with SimpleChatBot!")
        break
    
    # Add the user message to the message list
    messages.append(HumanMessage(user_input))
    
    # Process the chat request
    # ChatOpenAI is a wrapper around OpenAI's Chat Completions API
    response = llm.invoke(messages)
    
    # Add the AI message to the message list
    ai_message = response.content
    messages.append(AIMessage(ai_message))
    print(f"\nUser: {user_input}")
    print(f"\nAssistant: {ai_message}")

# Conversation history summary
print(f"\n📊 Conversation ended. Total messages exchanged: {len(messages) - 1}")  # Subtract 1 for system message

🤖 SimpleChatBot initialized!
Type 'exit' to end the conversation.
--------------------------------------------------

User: what is the name of the app?

Assistant: The name of the app is 'SimpleChatBot'. How can I assist you with your weather-related queries today?

👋 Goodbye! Thanks for chatting with SimpleChatBot!

📊 Conversation ended. Total messages exchanged: 2


# Task 2: Define Functions for Function Calling

In [14]:
# Function that returns supported locations
from langchain_core.tools import tool
import json

@tool
def get_supported_locations(country):
    """Get the supported locations in the given country"""
    print(f"Country={country}")
    
    if country.lower() == "us":
        return json.dumps({"locations": ["Philadelphia", "New York"]})
    elif country.lower() == "india":
        return json.dumps({"locations": ["Bangalore", "Chennai"]})
    else:
        return json.dumps({"locations": "None"})

In [15]:
# Function that returns localized recommendations
@tool
def get_localized_recommendations(location, category):
    """Get the local recommendations in the given location and category"""
    print(f"Location={location} and Category={category}")
    
    if "philadelphia" in location.lower():
        if "restaurants" in category.lower():
            return json.dumps({"recommendations": ["The Wayward", "The Victor Cafe"]})
    elif "new york" in location.lower():
        if "restaurants" in category.lower():
            return json.dumps({"recommendations": ["Dos Caminos", "Club A Steakhouse"]})
    elif "bangalore" in location.lower():
        if "restaurants" in category.lower():
            return json.dumps({"recommendations": ["Toit", "The Only Place", "Vidyarthi Bhavan"]})
        elif "events" in category.lower():
            return json.dumps({"recommendations": ["UB City Mall Events", "Phoenix MarketCity Events"]})
        elif "parks" in category.lower():
            return json.dumps({"recommendations": ["Cubbon Park", "Lalbagh Botanical Garden", "Bannerghatta National Park"]})
    elif "chennai" in location.lower():
        if "restaurants" in category.lower():
            return json.dumps({"recommendations": ["Murugan Idli Shop", "Hotel Saravana Bhavan"]})
        elif "events" in category.lower():
            return json.dumps({"recommendations": ["Express Avenue Events", "Phoenix MarketCity Events"]})
    else:
        return json.dumps({"recommendations": "None"})

In [16]:
# Bind the tools to the model
def initialize_langchain():
    # Process the request
    model = llm
    
    # Bind the tools to the model
    tools = [
        get_supported_locations,
        get_localized_recommendations
    ]
    global model_with_tools
    model_with_tools = model.bind_tools(tools)

# Task 3: Implement the Chat Processing Logic
This task implements the logic to process chat requests and handle function calls.
We create a process_chat_request function that sends chat requests, checks for tool calls,
and processes those tool calls by invoking the appropriate functions.


In [17]:
# Initialize variables and model
load_dotenv()
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)
messages = [
    SystemMessage(content="You are a helpful assistant for a chat application. The name of this app is 'SimpleChatBot'.")
]

In [18]:
# Implement the chat request processing function
def process_chat_request():
    """Process the chat request using function calling"""
    response = model_with_tools.invoke(messages)
    tool_calls = response.tool_calls
    if tool_calls:
        messages.append(response)
        for tool_call in tool_calls:
            function_name = tool_call["name"].lower()
            if function_name == "get_supported_locations":
                tool_msg = get_supported_locations.invoke(tool_call["args"])
            elif function_name == "get_localized_recommendations":
                tool_msg = get_localized_recommendations.invoke(tool_call["args"])
            else:
                tool_msg = json.dumps({"error": "Unknown function"})
            print(tool_msg)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call["id"],
                "content": tool_msg
            })
        response = model_with_tools.invoke(messages)
        print(response)
        return response
    return response

In [19]:
# Initialize and run the conversation loop
print("SimpleChatBot initialized!")
print("Type 'exit' to end the conversation.")
print("-" * 50)

initialize_langchain()

while True:
    user_input = input("\nYou: ").strip()
    if not user_input:
        continue
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("\nGoodbye! Thanks for chatting with SimpleChatBot!")
        break
    messages.append(HumanMessage(user_input))
    response = process_chat_request()
    ai_message = response.content
    messages.append(AIMessage(ai_message))
    print(f"\nAssistant: {ai_message}")

print(f"\nConversation ended. Total messages exchanged: {len(messages) - 1}")  # Subtract 1 for system message

SimpleChatBot initialized!
Type 'exit' to end the conversation.
--------------------------------------------------
Country=US
{"locations": ["Philadelphia", "New York"]}
Country=US
{"locations": ["Philadelphia", "New York"]}
content='The supported regions in the US are Philadelphia and New York.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 145, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-C5HOC0V02MiNx5eMe3Wbl6LiXEjds', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run--4a748502-0ff2-479b-9970-3ee892871068-0' usage_metadata={'input_tokens': 145, 'output_tokens': 14, 'total_tokens': 159, 'input_